In [306]:
import sqlite3

In [307]:
class Database:
    def __init__(self, db_name: str = "test.db") -> None:
        self.db_name = db_name

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            self.conn.rollback()
        self.conn.close()

    def __setattr__(self, name, value):
        if name == "run":
            if isinstance(value, str):
                self.show(value)
            elif isinstance(value, tuple):
                self.show(*value)
            return
        return super().__setattr__(name, value)

    def execute(
        self,
        sql: str,
        params: tuple[str, ...] | list[tuple[str, ...]] | None = None,
    ) -> list | None:
        try:
            if params is None:
                self.cursor.execute(sql)
            elif isinstance(params, tuple):
                self.cursor.execute(sql, params)
            else:
                self.cursor.executemany(sql, params)

            return self.cursor.fetchall()
        except sqlite3.Error as e:
            raise e

    def show(self, sql: str, params=None, limit=6) -> None:
        try:
            res = self.execute(sql, params)
        except sqlite3.Error as e:
            print(f"{e}\n")
            raise

        if self.cursor.description is None:
            print("Query successfully executed.\n")
            return

        columns = [desc[0] for desc in self.cursor.description]
        print(*columns, sep=" | ")

        if len(res) <= limit:
            print(*res, sep="\n")
        else:
            front_cnt = limit // 2
            back_cnt = limit - front_cnt

            print(*res[:front_cnt], sep="\n")
            print("...")
            print(*res[-back_cnt:], sep="\n")

        print(f"{len(res)} rows × {len(columns)} columns\n")

    def commit(self) -> None:
        self.conn.commit()

    def rollback(self) -> None:
        self.conn.rollback()

In [308]:
def create_tables():
    with Database() as db:
        db.execute("DROP TABLE IF EXISTS departments")
        db.execute("DROP TABLE IF EXISTS employees")

        db.execute(
            """
        CREATE TABLE departments (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            location TEXT,
            budget REAL
        )
        """
        )

        db.execute(
            """
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            age INTEGER,
            gender TEXT,
            address TEXT,
            salary REAL,
            hiredate TEXT,
            dept_id INTEGER,
            FOREIGN KEY (dept_id) REFERENCES departments(id)
        )
        """
        )

        departments_data = [
            ("인사부", "서울", 500000.0),
            ("개발부", "부산", 1000000.0),
            ("마케팅", "대전", 750000.0),
            ("영업부", "인천", 850000.0),
            ("연구소", "서울", 1200000.0),
            ("재무부", "부산", 900000.0),
            ("고객서비스", "대전", 600000.0),
            ("품질관리", "인천", 800000.0),
            ("물류센터", "서울", 700000.0),
            ("IT지원부", "광주", 950000.0),
        ]

        employees_data = [
            ("홍길동", 35, "남성", "서울", 45000.0, "2020-01-15", 1),
            ("김민수", 29, "남성", "부산", 65000.0, "2019-03-20", 2),
            ("이영희", 31, "여성", "부산", 55000.0, "2021-05-10", 2),
            ("Rodrigo", 42, "남성", "해외", 50000.0, "2018-11-30", 3),
            ("강지민", 27, "여성", "서울", 62000.0, "2022-02-15", None),
            ("김소진", 33, "여성", "인천", 48000.0, "2021-08-22", 4),
            ("Zhang", 38, "남성", "해외", 44000.0, "2023-01-10", None),
            ("신미나", 36, "여성", "서울", 70000.0, "2020-06-18", 5),
            ("Singh", 45, "남성", "해외", 68000.0, "2019-12-05", 5),
            ("Android123", None, None, None, 46000.0, "2022-09-30", 1),
            ("Gomez", 34, "남성", "해외", 58000.0, "2021-11-15", 5),
            ("한소희", 28, "여성", "대전", 52000.0, "2022-07-20", 7),
            ("홍길동", 99, "남성", "해외", 71000.0, "2019-08-25", 7),
            ("Wang", 32, "여성", "해외", 63000.0, "2020-04-12", 9),
            ("조성민", 37, "남성", "광주", 69000.0, "2021-03-08", 10),
            ("Chen", 35, "여성", "해외", 57000.0, "2022-05-15", None),
            ("Kumar", 39, "남성", "해외", 61000.0, "2021-07-22", None),
        ]

        db.execute(
            "INSERT INTO departments (name, location, budget) VALUES (?, ?, ?)",
            departments_data,
        )
        db.execute(
            "INSERT INTO employees (name, age, gender, address, salary, hiredate, dept_id) VALUES (?, ?, ?, ?, ?, ?, ?)",
            employees_data,
        )
        db.commit()


create_tables()

In [309]:
# SQL HOME, Intro, Syntax, Select
with Database() as db:
    db.run = "SELECT * FROM employees"
    db.run = "SELECT * FROM departments"
    db.run = "SELECT name, salary FROM employees"

id | name | age | gender | address | salary | hiredate | dept_id
(1, '홍길동', 35, '남성', '서울', 45000.0, '2020-01-15', 1)
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
(3, '이영희', 31, '여성', '부산', 55000.0, '2021-05-10', 2)
...
(15, '조성민', 37, '남성', '광주', 69000.0, '2021-03-08', 10)
(16, 'Chen', 35, '여성', '해외', 57000.0, '2022-05-15', None)
(17, 'Kumar', 39, '남성', '해외', 61000.0, '2021-07-22', None)
17 rows × 8 columns

id | name | location | budget
(1, '인사부', '서울', 500000.0)
(2, '개발부', '부산', 1000000.0)
(3, '마케팅', '대전', 750000.0)
...
(8, '품질관리', '인천', 800000.0)
(9, '물류센터', '서울', 700000.0)
(10, 'IT지원부', '광주', 950000.0)
10 rows × 4 columns

name | salary
('홍길동', 45000.0)
('김민수', 65000.0)
('이영희', 55000.0)
...
('조성민', 69000.0)
('Chen', 57000.0)
('Kumar', 61000.0)
17 rows × 2 columns



In [310]:
# SQL Select Distinct
with Database() as db:
    db.run = "SELECT DISTINCT gender FROM employees"
    db.run = "SELECT COUNT(DISTINCT gender) FROM employees"

gender
('남성',)
('여성',)
(None,)
3 rows × 1 columns

COUNT(DISTINCT gender)
(2,)
1 rows × 1 columns



In [311]:
# SQL Where
with Database() as db:
    db.run = "SELECT name, gender FROM employees WHERE gender = '남성'"
    db.run = "SELECT name, salary FROM employees WHERE salary > 60000"

name | gender
('홍길동', '남성')
('김민수', '남성')
('Rodrigo', '남성')
...
('홍길동', '남성')
('조성민', '남성')
('Kumar', '남성')
9 rows × 2 columns

name | salary
('김민수', 65000.0)
('강지민', 62000.0)
('신미나', 70000.0)
...
('Wang', 63000.0)
('조성민', 69000.0)
('Kumar', 61000.0)
8 rows × 2 columns



In [312]:
# SQL Order By
with Database() as db:
    db.run = "SELECT name, age FROM employees ORDER BY age DESC"
    db.run = "SELECT name, age FROM employees ORDER BY name ASC, age DESC"

name | age
('홍길동', 99)
('Singh', 45)
('Rodrigo', 42)
...
('한소희', 28)
('강지민', 27)
('Android123', None)
17 rows × 2 columns

name | age
('Android123', None)
('Chen', 35)
('Gomez', 34)
...
('한소희', 28)
('홍길동', 99)
('홍길동', 35)
17 rows × 2 columns



In [313]:
# SQL And, Or, Not
with Database() as db:
    db.run = "SELECT name, gender FROM employees WHERE name like '김%' AND gender = '남성'"
    db.run = "SELECT name, location FROM departments WHERE location = '서울' OR location = '부산'"
    db.run = (
        "SELECT name, location FROM departments WHERE Not location = '서울'"
    )
    # Combining AND, OR, and NOT
    db.run = "SELECT * FROM employees WHERE (name like '김%' OR name like '이%') AND NOT gender = '남성'"

name | gender
('김민수', '남성')
1 rows × 2 columns

name | location
('인사부', '서울')
('개발부', '부산')
('연구소', '서울')
('재무부', '부산')
('물류센터', '서울')
5 rows × 2 columns

name | location
('개발부', '부산')
('마케팅', '대전')
('영업부', '인천')
...
('고객서비스', '대전')
('품질관리', '인천')
('IT지원부', '광주')
7 rows × 2 columns

id | name | age | gender | address | salary | hiredate | dept_id
(3, '이영희', 31, '여성', '부산', 55000.0, '2021-05-10', 2)
(6, '김소진', 33, '여성', '인천', 48000.0, '2021-08-22', 4)
2 rows × 8 columns



In [314]:
# SQL Insert Into
with Database() as db:
    db.run = "SELECT * FROM departments WHERE name = '인턴부'"
    db.run = "INSERT INTO departments (name, location, budget) VALUES ('인턴부', '서울', 300000.0)"
    db.run = "SELECT * FROM departments WHERE name = '인턴부'"

    db.rollback()

    db.run = "SELECT count(*) FROM departments"
    db.run = """
        INSERT INTO departments (name, location, budget) 
        VALUES
            ('인턴부', '서울', 300000.0),
            ('인턴부2', '서울', 300000.0),
            ('인턴부3', '서울', 300000.0)
    """
    db.run = "SELECT count(*) FROM departments"

    db.rollback()

id | name | location | budget

0 rows × 4 columns

Query successfully executed.

id | name | location | budget
(11, '인턴부', '서울', 300000.0)
1 rows × 4 columns

count(*)
(10,)
1 rows × 1 columns

Query successfully executed.

count(*)
(13,)
1 rows × 1 columns



In [315]:
# SQL Null Values
with Database() as db:
    db.run = "SELECT * FROM employees WHERE address IS NULL"
    db.run = "SELECT * FROM employees WHERE address IS NOT NULL"

id | name | age | gender | address | salary | hiredate | dept_id
(10, 'Android123', None, None, None, 46000.0, '2022-09-30', 1)
1 rows × 8 columns

id | name | age | gender | address | salary | hiredate | dept_id
(1, '홍길동', 35, '남성', '서울', 45000.0, '2020-01-15', 1)
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
(3, '이영희', 31, '여성', '부산', 55000.0, '2021-05-10', 2)
...
(15, '조성민', 37, '남성', '광주', 69000.0, '2021-03-08', 10)
(16, 'Chen', 35, '여성', '해외', 57000.0, '2022-05-15', None)
(17, 'Kumar', 39, '남성', '해외', 61000.0, '2021-07-22', None)
16 rows × 8 columns



In [316]:
# SQL Update
with Database() as db:
    db.run = "SELECT * FROM employees WHERE name = '김민수'"
    db.run = "UPDATE employees SET salary = 70000 WHERE name = '김민수'"
    db.run = "SELECT * FROM employees WHERE name = '김민수'"

    db.rollback()

    db.run = "SELECT * FROM employees WHERE gender = '남성'"
    db.run = "UPDATE employees SET salary = 0 WHERE gender = '남성'"
    db.run = "SELECT * FROM employees WHERE gender = '남성'"

    db.rollback()

id | name | age | gender | address | salary | hiredate | dept_id
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
1 rows × 8 columns

Query successfully executed.

id | name | age | gender | address | salary | hiredate | dept_id
(2, '김민수', 29, '남성', '부산', 70000.0, '2019-03-20', 2)
1 rows × 8 columns

id | name | age | gender | address | salary | hiredate | dept_id
(1, '홍길동', 35, '남성', '서울', 45000.0, '2020-01-15', 1)
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
(4, 'Rodrigo', 42, '남성', '해외', 50000.0, '2018-11-30', 3)
...
(13, '홍길동', 99, '남성', '해외', 71000.0, '2019-08-25', 7)
(15, '조성민', 37, '남성', '광주', 69000.0, '2021-03-08', 10)
(17, 'Kumar', 39, '남성', '해외', 61000.0, '2021-07-22', None)
9 rows × 8 columns

Query successfully executed.

id | name | age | gender | address | salary | hiredate | dept_id
(1, '홍길동', 35, '남성', '서울', 0.0, '2020-01-15', 1)
(2, '김민수', 29, '남성', '부산', 0.0, '2019-03-20', 2)
(4, 'Rodrigo', 42, '남성', '해외', 0.0, '2018-11-30', 3)
...
(13, '홍길동', 99, '남성', '해

In [317]:
# SQL Delete
with Database() as db:
    db.run = "SELECT * FROM employees WHERE name = '김민수'"
    db.run = "DELETE FROM employees WHERE name = '김민수'"
    db.run = "SELECT * FROM employees WHERE name = '김민수'"

    db.rollback()

    db.run = "SELECT * FROM employees"
    db.run = "DELETE FROM employees"
    db.run = "SELECT * FROM employees"

    db.rollback()

    # Warning: This will not be rolled back
    # db.run = "DROP TABLE employees"

id | name | age | gender | address | salary | hiredate | dept_id
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
1 rows × 8 columns

Query successfully executed.

id | name | age | gender | address | salary | hiredate | dept_id

0 rows × 8 columns

id | name | age | gender | address | salary | hiredate | dept_id
(1, '홍길동', 35, '남성', '서울', 45000.0, '2020-01-15', 1)
(2, '김민수', 29, '남성', '부산', 65000.0, '2019-03-20', 2)
(3, '이영희', 31, '여성', '부산', 55000.0, '2021-05-10', 2)
...
(15, '조성민', 37, '남성', '광주', 69000.0, '2021-03-08', 10)
(16, 'Chen', 35, '여성', '해외', 57000.0, '2022-05-15', None)
(17, 'Kumar', 39, '남성', '해외', 61000.0, '2021-07-22', None)
17 rows × 8 columns

Query successfully executed.

id | name | age | gender | address | salary | hiredate | dept_id

0 rows × 8 columns



In [318]:
# SQL Select Top
with Database() as db:
    # Limit
    db.run = (
        "SELECT * FROM employees WHERE age > 40 ORDER BY salary DESC LIMIT 3"
    )

id | name | age | gender | address | salary | hiredate | dept_id
(13, '홍길동', 99, '남성', '해외', 71000.0, '2019-08-25', 7)
(9, 'Singh', 45, '남성', '해외', 68000.0, '2019-12-05', 5)
(4, 'Rodrigo', 42, '남성', '해외', 50000.0, '2018-11-30', 3)
3 rows × 8 columns



In [319]:
# SQL Min and Max
with Database() as db:
    db.run = "SELECT MAX(salary) FROM employees"
    db.run = "SELECT MIN(salary) FROM employees"
    db.run = "SELECT MAX(salary) as [Max Salary], MIN(salary) as [Min Salary], address FROM employees GROUP BY address"

MAX(salary)
(71000.0,)
1 rows × 1 columns

MIN(salary)
(44000.0,)
1 rows × 1 columns

Max Salary | Min Salary | address
(46000.0, 46000.0, None)
(69000.0, 69000.0, '광주')
(52000.0, 52000.0, '대전')
...
(70000.0, 45000.0, '서울')
(48000.0, 48000.0, '인천')
(71000.0, 44000.0, '해외')
7 rows × 3 columns



In [320]:
# SQL Count
with Database() as db:
    db.run = "SELECT COUNT(*) FROM employees"
    db.run = "SELECT COUNT(DISTINCT address) FROM employees"
    db.run = "SELECT COUNT(*) as [Number of Employees], address FROM employees GROUP BY address"

COUNT(*)
(17,)
1 rows × 1 columns

COUNT(DISTINCT address)
(6,)
1 rows × 1 columns

Number of Employees | address
(1, None)
(1, '광주')
(1, '대전')
...
(3, '서울')
(1, '인천')
(8, '해외')
7 rows × 2 columns



In [321]:
# SQL Sum
with Database() as db:
    db.run = "SELECT SUM(salary) FROM employees"
    db.run = "SELECT SUM(salary) FROM employees WHERE address = '서울'"
    db.run = "SELECT SUM(salary) as [Total Salary], address FROM employees GROUP BY address"

    db.run = "SELECT SUM(salary / 1000) as [Total Salary (in 1000s)], address FROM employees GROUP BY address"

SUM(salary)
(984000.0,)
1 rows × 1 columns

SUM(salary)
(177000.0,)
1 rows × 1 columns

Total Salary | address
(46000.0, None)
(69000.0, '광주')
(52000.0, '대전')
...
(177000.0, '서울')
(48000.0, '인천')
(472000.0, '해외')
7 rows × 2 columns

Total Salary (in 1000s) | address
(46.0, None)
(69.0, '광주')
(52.0, '대전')
...
(177.0, '서울')
(48.0, '인천')
(472.0, '해외')
7 rows × 2 columns



In [322]:
# SQL Avg
with Database() as db:
    db.run = "SELECT AVG(salary) / 1000 as [Average Salary (in 1000s)], address FROM employees GROUP BY address"
    db.run = "SELECT AVG(salary) FROM employees"
    db.run = "SELECT name, salary FROM employees WHERE salary > (SELECT AVG(salary) FROM employees)"

Average Salary (in 1000s) | address
(46.0, None)
(69.0, '광주')
(52.0, '대전')
...
(59.0, '서울')
(48.0, '인천')
(59.0, '해외')
7 rows × 2 columns

AVG(salary)
(57882.35294117647,)
1 rows × 1 columns

name | salary
('김민수', 65000.0)
('강지민', 62000.0)
('신미나', 70000.0)
...
('Wang', 63000.0)
('조성민', 69000.0)
('Kumar', 61000.0)
9 rows × 2 columns



In [323]:
# SQL Like
with Database() as db:
    db.run = "SELECT name, age, gender FROM employees WHERE name LIKE '김%'"
    db.run = "SELECT name, age, gender FROM employees WHERE name LIKE '_소_'"
    db.run = "SELECT name, age, gender FROM employees WHERE name LIKE '%민%'"
    db.run = "SELECT name, age, gender FROM employees WHERE name LIKE '_민%'"

name | age | gender
('김민수', 29, '남성')
('김소진', 33, '여성')
2 rows × 3 columns

name | age | gender
('김소진', 33, '여성')
('한소희', 28, '여성')
2 rows × 3 columns

name | age | gender
('김민수', 29, '남성')
('강지민', 27, '여성')
('조성민', 37, '남성')
3 rows × 3 columns

name | age | gender
('김민수', 29, '남성')
1 rows × 3 columns



In [324]:
# SQL Wildcards
with Database() as db:
    db.run = "SELECT name, address FROM employees WHERE name GLOB '*[a-z]*'"
    db.run = "SELECT name, address FROM employees WHERE name GLOB '*[0-9]*'"

name | address
('Rodrigo', '해외')
('Zhang', '해외')
('Singh', '해외')
...
('Wang', '해외')
('Chen', '해외')
('Kumar', '해외')
8 rows × 2 columns

name | address
('Android123', None)
1 rows × 2 columns



In [325]:
# SQL In
with Database() as db:
    db.run = (
        "SELECT name, address FROM employees WHERE address IN ('서울', '부산')"
    )
    db.run = "SELECT name, address FROM employees WHERE address NOT IN ('서울', '부산')"

    # 서브쿼리의 결과에 NULL이 포함되면 NOT IN의 연산결과는 항상 UNKNOWN이 된다.
    # id != A and id != B and id != NULL and ... -> UNKNOWN
    # 따라서 서브쿼리 연산 시 NULL을 제외해야 한다.
    # 다만 IN 연산 시에는 NULL을 제외하지 않아도 된다. (id = A or id = B or id = NULL or ...)
    db.run = """
        SELECT name, location 
        FROM departments 
        WHERE id NOT IN (
            SELECT distinct dept_id 
            FROM employees 
            WHERE dept_id IS NOT NULL
        )
    """

name | address
('홍길동', '서울')
('김민수', '부산')
('이영희', '부산')
('강지민', '서울')
('신미나', '서울')
5 rows × 2 columns

name | address
('Rodrigo', '해외')
('김소진', '인천')
('Zhang', '해외')
...
('조성민', '광주')
('Chen', '해외')
('Kumar', '해외')
11 rows × 2 columns

name | location
('재무부', '부산')
('품질관리', '인천')
2 rows × 2 columns



In [326]:
# SQL Between
with Database() as db:
    db.run = "SELECT name, salary FROM employees WHERE salary BETWEEN 50000 AND 60000 AND address IN ('서울', '부산')"

    db.run = "SELECT name, age, gender FROM employees WHERE name BETWEEN '김' AND '조'"
    db.run = "SELECT name, hiredate FROM employees WHERE hiredate NOT BETWEEN '2020-01-01' AND '2021-12-31'"

name | salary
('이영희', 55000.0)
1 rows × 2 columns

name | age | gender
('김민수', 29, '남성')
('이영희', 31, '여성')
('김소진', 33, '여성')
('신미나', 36, '여성')
4 rows × 3 columns

name | hiredate
('김민수', '2019-03-20')
('Rodrigo', '2018-11-30')
('강지민', '2022-02-15')
...
('한소희', '2022-07-20')
('홍길동', '2019-08-25')
('Chen', '2022-05-15')
9 rows × 2 columns



In [327]:
# SQL Aliases
with Database() as db:
    db.run = "SELECT name as [Full Name], salary * 12 as [Annual Salary] FROM employees"
    db.run = "SELECT name, age || ' ' || gender as [Info] FROM employees"

    db.run = """
        SELECT e.name as [Employee Name], d.name as [Department Name]
        FROM employees e, departments d
        ON e.dept_id = d.id
    """

Full Name | Annual Salary
('홍길동', 540000.0)
('김민수', 780000.0)
('이영희', 660000.0)
...
('조성민', 828000.0)
('Chen', 684000.0)
('Kumar', 732000.0)
17 rows × 2 columns

name | Info
('홍길동', '35 남성')
('김민수', '29 남성')
('이영희', '31 여성')
...
('조성민', '37 남성')
('Chen', '35 여성')
('Kumar', '39 남성')
17 rows × 2 columns

Employee Name | Department Name
('홍길동', '인사부')
('김민수', '개발부')
('이영희', '개발부')
...
('홍길동', '고객서비스')
('Wang', '물류센터')
('조성민', 'IT지원부')
13 rows × 2 columns



In [328]:
# SQL Joins, Inner Join
with Database() as db:
    # JOIN is the same as INNER JOIN
    db.run = "SELECT e.name as Employee, d.name as Department FROM employees e INNER JOIN departments d ON e.dept_id = d.id"

Employee | Department
('홍길동', '인사부')
('김민수', '개발부')
('이영희', '개발부')
...
('홍길동', '고객서비스')
('Wang', '물류센터')
('조성민', 'IT지원부')
13 rows × 2 columns



In [329]:
# SQL Left Join, Right Join, Full Join
with Database() as db:
    db.run = "SELECT e.name as Employee, d.name as Department FROM employees e LEFT JOIN departments d ON e.dept_id = d.id"
    db.run = "SELECT e.name as Employee, d.name as Department FROM employees e RIGHT JOIN departments d ON e.dept_id = d.id"
    db.run = "SELECt e.name as Employee, d.name as Department FROM employees e FULL JOIN departments d ON e.dept_id = d.id"

Employee | Department
('홍길동', '인사부')
('김민수', '개발부')
('이영희', '개발부')
...
('조성민', 'IT지원부')
('Chen', None)
('Kumar', None)
17 rows × 2 columns

Employee | Department
('홍길동', '인사부')
('김민수', '개발부')
('이영희', '개발부')
...
('조성민', 'IT지원부')
(None, '재무부')
(None, '품질관리')
15 rows × 2 columns

Employee | Department
('홍길동', '인사부')
('김민수', '개발부')
('이영희', '개발부')
...
('Kumar', None)
(None, '재무부')
(None, '품질관리')
19 rows × 2 columns



In [330]:
# SQL Self Join
with Database() as db:
    db.run = """
        SELECT e1.name as [Employee], e2.name as [Manager]
        FROM employees e1, employees e2
        WHERE e1.dept_id = e2.dept_id AND e1.id < e2.id
    """

Employee | Manager
('홍길동', 'Android123')
('김민수', '이영희')
('신미나', 'Gomez')
('신미나', 'Singh')
('Singh', 'Gomez')
('한소희', '홍길동')
6 rows × 2 columns



In [331]:
# SQL Union
with Database() as db:
    # UNION은 중복을 제거한다. (출력 결과를 보니 중복 제거 과정에서 정렬을 수행하는 것으로 보인다.)
    db.run = "SELECT name FROM employees UNION SELECT name FROM departments"
    # UNION ALL은 중복을 제거하지 않는다.
    db.run = (
        "SELECT name FROM employees UNION ALL SELECT name FROM departments"
    )

    db.run = """
        SELECT name as [직원 혹은 부서 이름], address as 지역 FROM employees
        WHERE gender = '남성'
        UNION ALL
        SELECT name, location FROM departments
        ORDER BY address
    """

name
('Android123',)
('Chen',)
('Gomez',)
...
('품질관리',)
('한소희',)
('홍길동',)
26 rows × 1 columns

name
('홍길동',)
('김민수',)
('이영희',)
...
('품질관리',)
('물류센터',)
('IT지원부',)
27 rows × 1 columns

직원 혹은 부서 이름 | 지역
('조성민', '광주')
('IT지원부', '광주')
('마케팅', '대전')
...
('Gomez', '해외')
('홍길동', '해외')
('Kumar', '해외')
19 rows × 2 columns



In [332]:
# SQL Group By
with Database() as db:
    db.run = "SELECT address, COUNT(*) as [Number of Employees], AVG(salary) as [Average Salary] FROM employees GROUP BY address"

    db.run = """
        SELECT d.name as [Department], COUNT(e.id) as [Number of Employees]
        FROM departments d
            LEFT JOIN employees e
                ON d.id = e.dept_id
        GROUP BY d.id
        ORDER BY [Number of Employees] DESC
    """

address | Number of Employees | Average Salary
(None, 1, 46000.0)
('광주', 1, 69000.0)
('대전', 1, 52000.0)
...
('서울', 3, 59000.0)
('인천', 1, 48000.0)
('해외', 8, 59000.0)
7 rows × 3 columns

Department | Number of Employees
('연구소', 3)
('인사부', 2)
('개발부', 2)
...
('IT지원부', 1)
('재무부', 0)
('품질관리', 0)
10 rows × 2 columns



In [333]:
# SQL Having
with Database() as db:
    # SQL 표준에는 HAVING 절에서 별칭을 사용할 수 없다고 되어 있지만, 특정 DBMS에서는 사용이 가능하다고 함.
    db.run = """
        SELECT d.name as [Department], COUNT(e.id) as [Number of Employees]
        FROM departments d
            LEFT JOIN employees e
                ON d.id = e.dept_id
        GROUP BY d.id
        HAVING [Number of Employees] > 1
        ORDER BY [Number of Employees] DESC
    """

Department | Number of Employees
('연구소', 3)
('인사부', 2)
('개발부', 2)
('고객서비스', 2)
4 rows × 2 columns



In [335]:
# SQL Exists
with Database() as db:
    db.run = """
        SELECT name, location
        FROM departments d
        WHERE EXISTS (
            SELECT *
            FROM employees e
            WHERE e.dept_id = d.id
                AND e.salary > 60000
        )
    """

name | location
('개발부', '부산')
('연구소', '서울')
('고객서비스', '대전')
('물류센터', '서울')
('IT지원부', '광주')
5 rows × 2 columns



In [345]:
# SQL All
with Database() as db:
    # ALL은 'operator ALL' 형태로 사용된다. (이 쌍이 하나의 연산자로 취급된다.)
    # 서브쿼리의 결과 중 모든 값이 조건을 만족할 때 참이 된다.

    sql = """
        SELECT name, salary
        FROM employees
        WHERE salary > ALL (
            SELECT salary
            FROM employees
            WHERE dept_id = 2
        )
    """

    # SQLite에서는 ALL 연산자를 지원하지 않는다.
    # 동일한 결과를 얻는 쿼리는 다음과 같다.

    db.run = """
        SELECT name, salary
        FROM employees
        WHERE salary > (
            SELECT MAX(salary)
            FROM employees
            WHERE dept_id = 2
        )
    """

name | salary
('신미나', 70000.0)
('Singh', 68000.0)
('홍길동', 71000.0)
('조성민', 69000.0)
4 rows × 2 columns

